# **Projekt: Fehlererkennung in Getrieben**
## Netztyp 2: FFT-Net

#### **Vorbereitung**

In [107]:
# import libraries
import numpy as np
import os
from pandas import DataFrame, concat
from pathlib import Path
from sklearn.utils.class_weight import compute_class_weight

# module imports from /src
if Path.cwd().stem == "notebooks":
    os.chdir( Path.cwd().parent)

import src.data_loader as dl
import src.visualization as vis
import src.model_constructor as mc
import src.input_preparation as ip

In [108]:
# set the source directory for the preprocessed data to use
SOURCE: str = "c556197b-3225-45f7-8e03-68b4fb8a189d"

# setup a single index for control purposes
CONTROL_INDEX: int = 0

In [ ]:
# setup system and check the number of cpu cores and gpus available
mc.system_setup()

#### **Laden der Datensätze**

In [ ]:
# load data from folder and split in training and evaluation data
data_path = Path().cwd() / "data" / "processed" / SOURCE
development_data, evaluation_data = dl.load_all_datasets(data_path)

In [ ]:
# visualize one random dataset for data validation
vis.plot_column_data(development_data[CONTROL_INDEX],
                             development_data[CONTROL_INDEX].columns)

#### **Das Neuronale Netz**

In [ ]:
full_development_dataset = ip.concatenate_datasets(development_data)

training_samples_dict = ip.split_data(
    dataframe=full_development_dataset,
    data_columns=[
        "vibration_1_magnitude",
        # "vibration_2_magnitude",
        # "vibration_3_magnitude",
    ],
)

print(training_samples_dict["labels"].shape)
print(training_samples_dict["samples"].shape)

In [ ]:
ip.check_data(training_samples_dict)

In [114]:
# model creation parameters
N_HIDDEN_LAYERS: int = 4
L2: float = 1e-3 # 0.001
DROPOUT: float = 0.3
NEGATIVE_SLOPE: float = 0.3

# model compilation parameters
LEARNING_RATE: float = 1e-4

# model training parameters
BATCH_SIZE: int = 64
EPOCHS: int = 20
VALIDATION_SPLIT: float = 0.1

# manual changes to class weights
training_samples_dict["class_weights"] = {
    0: 0.7,  # none (Klasse 0) wird weniger stark gewichtet
    1: 2.5,  # slight (Klasse 1) wird stärker gewichtet
    2: 1.5,  # moderate (Klasse 2) wird leicht erhöht
    3: 1.2,  # significant (Klasse 3) bleibt gleich
    4: 0.7   # strong (Klasse 4) wird weniger stark gewichtet
}

In [ ]:
model = mc.construct_fft_net_model(n_hidden_layers=N_HIDDEN_LAYERS,
                                   training_samples_dict=training_samples_dict,
                                   l2=L2,
                                   dropout=DROPOUT,
                                   negative_slope=NEGATIVE_SLOPE)

In [ ]:
mc.compile_model(model=model,
                 learning_rate=LEARNING_RATE)


history = mc.train_model(model=model,
                         samples_dict=training_samples_dict,
                         epochs=EPOCHS,
                         batch_size=BATCH_SIZE,
                         validation_split=VALIDATION_SPLIT,
                         use_early_stopping=True)

#### **Bewertung des Neuronalen Netzes**

In [ ]:
# visualize training metrics
vis.plot_history(history, metrics=["loss", "accuracy", "precision", "recall"])

In [ ]:
full_evaluation_dataset = ip.concatenate_datasets(evaluation_data)

test_samples_dict = ip.split_data(dataframe = full_evaluation_dataset, data_columns = [
        "vibration_1_magnitude",
        # "vibration_2_magnitude",
        # "vibration_3_magnitude",
    ],
)

print(test_samples_dict["labels"].shape)
print(test_samples_dict["samples"].shape)

In [ ]:
ip.check_data(test_samples_dict)

In [ ]:
# evaluate model
evaluation = mc.evaluate(model=model, test_samples_dict=test_samples_dict)
display(evaluation)

### Confusion Matrix
Die Confusion Matrix zeigt, wie gut dein Modell zwischen den verschiedenen Klassen unterscheidet. Jede Zelle stellt die Anzahl der Samples dar, die einer bestimmten Klasse zugeordnet wurden (wahre Labels) und wie das Modell diese Klasse vorhergesagt hat (vorhergesagte Labels).



In [ ]:
# predictions by model
true_labels, predicted_labels = mc.predict(model=model, test_samples_dict=test_samples_dict)

print(type(test_samples_dict["encoder"].classes_))

# plot confusion matrix
vis.plot_confusion_matrix(true_labels=true_labels,
                          predicted_labels=predicted_labels,
                          class_names=test_samples_dict["encoder"].classes_.tolist()
                          )